In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df = df.dropna()

In [4]:
X = df.drop('label', axis = 1)

In [5]:
y = df['label']

In [6]:
X.shape

(18285, 4)

In [7]:
import tensorflow as tf
from keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM, Dense

2023-10-22 15:36:26.231222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-22 15:36:27.983188: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/harsh/miniconda3/envs/tf/lib/:/home/harsh/miniconda3/envs/tf/lib/
2023-10-22 15:36:27.986398: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/harsh/miniconda3/envs/tf/lib/:/home/harsh/minicond

In [8]:
voc_size = 5000

In [9]:
messages = X.copy()

In [10]:
messages.reset_index(inplace=True)

In [11]:
import nltk
import re
from nltk.corpus import stopwords

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/harsh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [14]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3252, 297, 4035, 3592, 3534, 2743, 281, 2803, 4653, 2749],
 [2627, 3711, 2713, 2396, 1655, 3555, 3474],
 [210, 792, 1486, 3487],
 [2655, 4825, 1082, 814, 4462, 1364],
 [1643, 1655, 4924, 2387, 990, 3180, 1655, 4260, 2506, 4803],
 [1737,
  4506,
  971,
  4758,
  1820,
  1583,
  1348,
  4263,
  2173,
  870,
  204,
  3879,
  760,
  2083,
  3474],
 [4938, 4027, 4792, 4247, 1615, 2823, 4199, 801, 1994, 2049, 4555],
 [4287, 540, 3685, 1921, 3206, 2537, 1583, 2828, 1994, 2049, 4555],
 [4569, 3280, 4601, 1974, 4387, 2974, 3802, 894, 1583, 3214],
 [3482, 4017, 1733, 286, 2166, 1439, 2652, 2946],
 [1179, 348, 4219, 2916, 1202, 15, 993, 2555, 1209, 3349, 1451],
 [814, 4690, 3534, 2974, 1583, 3206],
 [3053, 3853, 2565, 2881, 2827, 3428, 1935, 14, 3936],
 [2746, 3620, 770, 4697, 708, 3481, 127, 1994, 2049, 4555],
 [2521, 1006, 3178, 936, 4586, 1994, 2049, 4555],
 [3793, 4858, 4591, 362, 2430, 4643, 297, 1432, 4634, 3689],
 [1957, 3654, 3711],
 [336, 2716, 515, 3933, 1583, 330, 2266, 3474],
 [2863

In [15]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2803 4653 2749]
 [   0    0    0 ... 1655 3555 3474]
 [   0    0    0 ...  792 1486 3487]
 ...
 [   0    0    0 ... 1994 2049 4555]
 [   0    0    0 ...  271 2738 3490]
 [   0    0    0 ... 4027 1661 2670]]


In [16]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3252,
        297, 4035, 3592, 3534, 2743,  281, 2803, 4653, 2749], dtype=int32)

In [17]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


2023-10-22 15:36:46.713781: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 15:36:46.777421: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 15:36:46.777618: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 15:36:46.779935: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
len(embedded_docs),y.shape


(18285, (18285,))

In [19]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [20]:
X_final.shape,y_final.shape


((18285, 20), (18285,))

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [25]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=32)

Epoch 1/100
383/383 [==============================] - 4s 10ms/step - loss: 7.5553e-07 - accuracy: 1.0000 - val_loss: 1.1576 - val_accuracy: 0.9027
Epoch 2/100
383/383 [==============================] - 3s 9ms/step - loss: 6.3661e-07 - accuracy: 1.0000 - val_loss: 1.1938 - val_accuracy: 0.9049
Epoch 3/100
383/383 [==============================] - 3s 9ms/step - loss: 5.4214e-07 - accuracy: 1.0000 - val_loss: 1.1900 - val_accuracy: 0.9042
Epoch 4/100
383/383 [==============================] - 3s 9ms/step - loss: 4.1459e-07 - accuracy: 1.0000 - val_loss: 1.2031 - val_accuracy: 0.9031
Epoch 5/100
383/383 [==============================] - 4s 9ms/step - loss: 3.3799e-07 - accuracy: 1.0000 - val_loss: 1.2198 - val_accuracy: 0.9037
Epoch 6/100
383/383 [==============================] - 3s 9ms/step - loss: 2.7216e-07 - accuracy: 1.0000 - val_loss: 1.2336 - val_accuracy: 0.9034
Epoch 7/100
383/383 [==============================] - 4s 9ms/step - loss: 2.3237e-07 - accuracy: 1.0000 - val_loss: 